In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import SAUCIE
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from consolidated_runs import run_simulations
from utils import gt_benchmark

sys.path.append(os.getcwd())

### Generate SERGIO and Imputation (only) data for benchmarking & runtime analysis

In [4]:
datasets = range(3, 4) # DS3
run_simulations(datasets,
                #setting to true will run SERGIO to create clean & noisy data. Must be set to true to run GENIE3 on clean & noisy data
                sergio=True,
                saucie=False, 
                scScope=False, 
                deepImpute=False, 
                magic=True, 
                genie=False,
                arboreto=False,
                roc=False,
                precision_recall_k=False
            )

  0%|          | 0/1 [00:00<?, ?it/s]

---> Running SERGIO on DS3
Start simulating new level
There are 2 genes to simulate in this layer
Done with current level
Start simulating new level
There are 3 genes to simulate in this layer
Done with current level
Start simulating new level
There are 5 genes to simulate in this layer
Done with current level
Start simulating new level
There are 9 genes to simulate in this layer
Done with current level
Start simulating new level
There are 26 genes to simulate in this layer
Done with current level
Start simulating new level
There are 82 genes to simulate in this layer
Done with current level
Start simulating new level
There are 1073 genes to simulate in this layer
Done with current level
---> Running MAGIC on DS3
./imputations/DS3/DS6_clean.npy ./imputations/DS3/DS6_45.npy
Calculating MAGIC...
  Running MAGIC on 2700 cells and 1193 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 7.74 seconds.
    Calculating KNN search...
    Calculated

100%|██████████| 1/1 [31:44<00:00, 1904.71s/it]


In [29]:
algorithms = ['Pearson', 'GENIE3']

data_path = './imputations/DS3/'
datasets = {
    'sergio' : 'DS6_45.npy',
    'magic_2' : 'yhat_MAGIC_t_2.npy',
    'magic_7' : 'yhat_MAGIC_t_7.npy',
    'magic_default' : 'yhat_MAGIC_t_auto.npy',
}

In [49]:
from Pearson.pearson import Pearson
from GENIE3.GENIE3 import GENIE3
target_file = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'
y = np.transpose(np.load(data_path + '/DS6_45.npy'))

In [40]:
vim_genie = GENIE3(y, nthreads=12, ntrees=100)       
gt, rescaled_vim = gt_benchmark(vim_genie, target_file)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads
Elapsed time: 88.10 seconds


100%|██████████| 1133/1133 [00:00<00:00, 545221.02it/s]


In [50]:
test_pearson

,0,1,2,3,4,5,6,7,8,9,...,2690,2691,2692,2693,2694,2695,2696,2697,2698,2699
0,0.000000,0.878601,0.769895,0.976397,0.924053,0.975616,0.857518,0.952817,0.944047,0.986210,...,0.924950,0.973663,0.954979,0.892936,0.932568,0.934366,0.930085,0.957206,0.927748,0.880889
1,0.878601,0.000000,0.487521,0.900275,0.693416,0.871493,0.988259,0.840618,0.828303,0.859367,...,0.821898,0.825637,0.758673,0.653123,0.916933,0.736343,0.880233,0.820339,0.880554,0.902021
2,0.769895,0.487521,0.000000,0.704156,0.914339,0.831693,0.449082,0.738152,0.871058,0.817506,...,0.875554,0.761463,0.829239,0.885601,0.729367,0.886081,0.778652,0.853729,0.743333,0.708188
3,0.976397,0.900275,0.704156,0.000000,0.893268,0.961260,0.873353,0.967072,0.918286,0.971927,...,0.901087,0.977945,0.942536,0.834291,0.901728,0.888811,0.929466,0.919271,0.888694,0.844008
4,0.924053,0.693416,0.914339,0.893268,0.000000,0.941169,0.659189,0.900600,0.940285,0.950032,...,0.923506,0.936312,0.961368,0.938699,0.843596,0.958221,0.881753,0.943020,0.855486,0.791605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0.934366,0.736343,0.886081,0.888811,0.958221,0.943792,0.722807,0.873790,0.937502,0.958414,...,0.908772,0.926002,0.951562,0.974411,0.907489,0.000000,0.867670,0.978620,0.927446,0.862752
2696,0.930085,0.880233,0.778652,0.929466,0.881753,0.961560,0.843088,0.917683,0.953670,0.938347,...,0.963859,0.900270,0.881276,0.798391,0.910728,0.867670,0.000000,0.907867,0.880039,0.890354
2697,0.957206,0.820339,0.853729,0.919271,0.943020,0.969894,0.807258,0.896818,0.960898,0.974968,...,0.939388,0.932504,0.944109,0.948904,0.953701,0.978620,0.907867,0.000000,0.960618,0.919448
2698,0.927748,0.880554,0.743333,0.888694,0.855486,0.934366,0.886134,0.833785,0.922128,0.937565,...,0.896830,0.874314,0.867174,0.888256,0.984568,0.927446,0.880039,0.960618,0.000000,0.965418


In [48]:
geniedf = pd.DataFrame(vim_genie)
for index, row in geniedf.iterrows():
    if row[index] != 0:
        print(index, row[index])